In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import os
import ipywidgets as widgets
from IPython.display import display


def actualizar_datos(_):
    with psycopg2.connect(**conn_params) as conn:
        global df
        df = pd.read_sql(query, conn)
    print("Datos actualizados:")
    print(df.head())

boton_actualizar = widgets.Button(description="Actualizar Datos")
boton_actualizar.on_click(actualizar_datos)

display(boton_actualizar)

# Conectar y cargar datos en un DataFrame
conn_params = {
    "host": "localhost",
    "database": "tasks_db",
    "user": "tasks_user",
    "password": "123456"
}

query = "SELECT * FROM Task"
df = pd.read_sql_query(query, psycopg2.connect(**conn_params))


# Mostrar datos cargados
print("Datos de las tareas:")
print(df.head())


# Función: Gráfico de tareas completadas vs pendientes
def plot_task_distribution(df):
    completed_count = df['completed'].sum()
    pending_count = len(df) - completed_count

    labels = ['Completadas', 'Pendientes']
    values = [completed_count, pending_count]

    plt.figure(figsize=(6, 6))
    plt.pie(values, labels=labels, autopct='%1.1f%%', startangle=140, colors=['#4CAF50', '#F44336'])
    plt.title('Distribución de Tareas')
    plt.savefig('../static/images/distribution_of_tasks.png')  # Guardar gráfico
    plt.show()

    print(f"Tareas Completadas: {completed_count}")
    print(f"Tareas Pendientes: {pending_count}")


# Función: Gráfico de tareas creadas por día
def plot_tasks_per_day(df):
    if 'date_added' in df.columns:
        df['date_added'] = pd.to_datetime(df['date_added'])
        tasks_per_day = df.groupby(df['date_added'].dt.date).size()

        plt.figure(figsize=(10, 6))
        tasks_per_day.plot(kind='bar', color='skyblue')
        plt.title('Tareas Creadas por Día')
        plt.xlabel('Fecha')
        plt.ylabel('Número de Tareas')
        plt.xticks(rotation=45)
        plt.tight_layout()
        plt.savefig('../static/images/tasks_per_day.png')  # Guardar gráfico
        plt.show()
    else:
        print("La columna 'date_added' no está disponible en los datos.")

# Función: Distribución de la longitud de los títulos
def plot_title_length_distribution(df):
    df['title_length'] = df['title'].apply(len)

    plt.hist(df['title_length'], bins=10, color='purple', alpha=0.7)
    plt.title('Distribución de la Longitud de los Títulos')
    plt.xlabel('Longitud del Título')
    plt.ylabel('Frecuencia')
    plt.show()


def plot_completion_progress_over_time(df):
    # Verificar columnas necesarias
    required_columns = ['date_added', 'completed']
    if not all(col in df.columns for col in required_columns):
        print(f"Columnas faltantes: {[col for col in required_columns if col not in df.columns]}")
        return
    
    try:
        # Convertir fecha y manejar errores
        df['date_added'] = pd.to_datetime(df['date_added'], errors='coerce')
        df = df.dropna(subset=['date_added'])  # Eliminar filas con fechas inválidas
        
        if df.empty:
            print("No hay datos válidos después de filtrar fechas.")
            return

        # Agrupar y calcular progreso
        progress_over_time = df.groupby(df['date_added'].dt.date)['completed'].mean() * 100
        
        if progress_over_time.empty:
            print("No hay datos para graficar.")
            return

        # Crear gráfico
        plt.figure(figsize=(10, 6))
        progress_over_time.plot(marker='o', linestyle='-', color='teal', label='Progreso (%)')
        
        # Configuración del gráfico
        plt.title('Progreso de Tareas a lo Largo del Tiempo', fontsize=16, fontweight='bold')
        plt.xlabel('Fecha', fontsize=12)
        plt.ylabel('Progreso (%)', fontsize=12)
        plt.xticks(rotation=45, fontsize=10)
        plt.yticks(fontsize=10)
        plt.grid(True, linestyle='--', alpha=0.5)
        plt.legend()
        plt.tight_layout()
        
        # Crear directorio si no existe
        os.makedirs('../static/images', exist_ok=True)
        
        # Guardar y mostrar
        plt.savefig('../static/images/progress_over_time.png')
        plt.show()
        
    except Exception as e:
        print(f"Error al generar el gráfico: {str(e)}")

# Generar gráficos
plot_task_distribution(df)
plot_tasks_per_day(df)
plot_title_length_distribution(df)
plot_completion_progress_over_time(df)







Button(description='Actualizar Datos', style=ButtonStyle())

C:\Users\sebax\AppData\Local\Temp\ipykernel_23236\637495999.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, psycopg2.connect(**conn_params))
c:\proyectTareas\venv\Lib\site-packages\matplotlib\axes\_axes.py:3343: RuntimeWarning: invalid value encountered in divide
  x = x / sx


Datos de las tareas:
Empty DataFrame
Columns: [id, title, description, completed, date_added, user_id]
Index: []


ValueError: cannot convert float NaN to integer

ValueError: need at least one array to concatenate

<Figure size 600x600 with 1 Axes>